<a href="https://colab.research.google.com/github/D2HStack/Best-README-Template/blob/master/crypto_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
############## GET THE MARK PRICE ##########################
# Import packages
import requests

# Access secrets
from google.colab import userdata
PUBLIC_KEY=userdata.get('PUBLIC_KEY')
PRIVATE_KEY=userdata.get('PRIVATE_KEY')

# Constants
API_URL="https://testnet.binancefuture.com"
HEADERS={"X-MBX-APIKEY": PUBLIC_KEY}

# Request parameters
endpoint="/fapi/v1/premiumIndex"
params={'symbol': 'BTCUSDT'}

# Make the request
try:
    response=requests.get(API_URL + endpoint, params=params, headers=HEADERS)
    print(response.json())
except Exception as e:
    print(e)

In [ ]:
############## GET THE ORDER BOOK ##########################
# Import packages
import requests

# Access secrets
from google.colab import userdata
PUBLIC_KEY=userdata.get('PUBLIC_KEY')
PRIVATE_KEY=userdata.get('PRIVATE_KEY')

# Constants
API_URL="https://testnet.binancefuture.com"
HEADERS={"X-MBX-APIKEY": PUBLIC_KEY}

# Request parameters
endpoint="/fapi/v1/depth"
params={'symbol': 'BTCUSDT',
        'limit': 10,
}

# Make the request
try:
    response=requests.get(API_URL + endpoint, params=params, headers=HEADERS).json()
    bids=response['bids']
    asks=response['asks']
    spread=float(asks[0][0]) - float(bids[0][0])
    print(bids)
    print(asks)
    print(asks[0][0])
    print(bids[0][0])
    print(spread)
except Exception as e:
    print(e)

In [ ]:
############## CANDLESTICK GRAPH ##########################
# Import packages
import requests
import time
import datetime
import plotly.graph_objects as go
import pandas as pd

# API timeframe 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# Access secrets
from google.colab import userdata
PUBLIC_KEY=userdata.get('PUBLIC_KEY')
PRIVATE_KEY=userdata.get('PRIVATE_KEY')

# Constants
API_URL="https://testnet.binancefuture.com"
HEADERS={"X-MBX-APIKEY": PUBLIC_KEY}

# Request parameters
endpoint="/fapi/v1/klines"
startTime= "2024-02-01 00:00:00" # Enter the start time as 2024-02-14 00:00:00
endTime= "2024-02-11 00:00:00" # Enter the start time as 2024-02-14 00:00:00
startTimeInt=(datetime.datetime.fromisoformat(startTime).timestamp())*1000
endTimeInt=(datetime.datetime.fromisoformat(endTime).timestamp())*1000
# endTime=int(time.time()*1000) # Time Now
params={
    'symbol': 'BTCUSDT',
    'interval': '1d',
    'startTime': startTimeInt,
    'endTime': endTimeInt,
}


try:
    # Make the request
    response=requests.get(API_URL + endpoint, params=params, headers=HEADERS)
    # print(response.json())

    # Build the dataframe
    df=pd.DataFrame(response.json(), columns=['date', 'open', 'high', 'low', 'close','a','b','c','d','e','f','g'])
    df['date']=pd.to_datetime(df['date'], unit='ms')

    # Plot the candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()

except Exception as e:
    print(e)

In [ ]:
############## MARKET ORDER ##########################
# Import packages
import requests
import time
import hmac # signature
from urllib.parse import urlencode # signature
import hashlib # signature

# Access secrets
from google.colab import userdata
PUBLIC_KEY=userdata.get('PUBLIC_KEY')
PRIVATE_KEY=userdata.get('PRIVATE_KEY')

# Constants
API_URL="https://testnet.binancefuture.com"
HEADERS={"X-MBX-APIKEY": PUBLIC_KEY}

# Request parameters
endpoint="/fapi/v1/order"
params={
    'symbol': 'BTCUSDT',
    'side': 'SELL',
    'type': 'MARKET',
    'quantity': 0.01,
    'timestamp': int(time.time()*1000), # Time Now
}
# Sign the transaction
signature=hmac.new(PRIVATE_KEY.encode(), urlencode(params).encode(), hashlib.sha256).hexdigest()
params['signature']=signature

try:
    # Make request
    response=requests.post(API_URL + endpoint, params=params, headers=HEADERS)
    print(response.json())
except Exception as e:
    print(e)



In [ ]:
############## MARKET PRICE WITH WEBSOCKET ##########################
# Import packages
import requests
import websocket
import time
import datetime
import json

# Access secrets
from google.colab import userdata
PUBLIC_KEY=userdata.get('PUBLIC_KEY')
PRIVATE_KEY=userdata.get('PRIVATE_KEY')

# Constants
WSS_URL="wss://stream.binancefuture.com/ws"

# Definition of websocket functions
def on_open(ws):
  print("Binance Market Websocket connection opened")
def on_close(ws):
  print("Binance Market Websocket connection closed")
def on_error(ws, msg: str):
  print("Binance Market Websocket connection error: {}".format(msg))
def on_message(ws, msg: str):
  print('message in')
  data=json.loads(msg)
  if 'e' in data:
    print("At {}, the Mark price for {} is {}".format(datetime.datetime.fromtimestamp(data['E']/1000).strftime("%m/%d/%Y, %H:%M:%S"), data['s'], data['p']))

# Open the market websocket
stream_name="btcusdt@markPrice"
market_ws=websocket.WebSocketApp(WSS_URL + "/" + stream_name, on_open=on_open, on_close=on_close, on_error=on_error, on_message=on_message)
while True:
  try:
    market_ws.run_forever()
  except Exception as e:
    print("Binance Market Websocket error in run_forever() method: {}".format(e))
  time.sleep(2)



In [ ]:
############## USER WEBSOCKET ##########################
# Import packages
import requests
import websocket
import time
import json

# Access secrets
from google.colab import userdata
PUBLIC_KEY=userdata.get('PUBLIC_KEY')
PRIVATE_KEY=userdata.get('PRIVATE_KEY')

# Constants
API_URL="https://testnet.binancefuture.com"
WSS_URL="wss://stream.binancefuture.com/ws"
HEADERS={"X-MBX-APIKEY": PUBLIC_KEY}

# Get the listen key
endpoint="/fapi/v1/listenKey"
params={
    'timestamp': int(time.time()*1000), # Time Now
}
try:
    # Make request
    listen_key=requests.post(API_URL + endpoint, params=params, headers=HEADERS).json()['listenKey']
    print(listen_key)
except Exception as e:
    print(e)

# Definition of websocket functions
def on_open(ws):
  print("Binance User Websocket connection opened")
def on_close(ws):
  print("Binance User Websocket connection closed")
def on_error(ws, msg: str):
  print("Binance User Websocket connection error: {}".format(msg))
def on_message(ws, msg: str):
  print('message in')
  data=json.loads(msg)
  if 'e' in data:
    print(data['e'])

# Open the user websocket
user_ws=websocket.WebSocketApp(WSS_URL + "/" + listen_key, on_open=on_open, on_close=on_close, on_error=on_error, on_message=on_message)
while True:
  try:
    user_ws.run_forever()
  except Exception as e:
    print("Binance User Websocket error in run_forever() method: {}".format(e))
  time.sleep(2)